In [1]:
# ────────────────────────────────
# 📦 Importación de librerías
# ────────────────────────────────
import json
import pandas as pd
import numpy as np
import re
import os

# ────────────────────────────────
# 📁 Configuración inicial
# ────────────────────────────────
carpeta = r"C:\Users\yanin\OneDrive\Desktop\etl\data_lake\raw\google\metadata-sitios-20250406T191417Z-001\metadata-sitios"
output_path = r"c:\Users\yanin\OneDrive\Desktop\etl\data_lake\trusted\google\restaurantes_CA_WY.parquet"
lista_df = []

# ────────────────────────────────
# 🧩 Funciones auxiliares
# ────────────────────────────────
def extraer_estado(address):
    if isinstance(address, str):
        match = re.search(r',\s*([A-Z]{2})\s*\d{5}', address)
        if match:
            return match.group(1)
    return np.nan

def extraer_ciudad(address):
    if isinstance(address, str):
        partes = address.split(",")
        if len(partes) >= 3:
            ciudad = partes[-2].strip()
            return ciudad
    return np.nan

# ────────────────────────────────
# 🔁 Procesamiento de cada archivo JSON
# ────────────────────────────────
for archivo in os.listdir(carpeta):
    if archivo.endswith(".json"):
        file_path = os.path.join(carpeta, archivo)
        print(f"📂 Procesando: {archivo}")

        # Intentar cargar el archivo JSON
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)
        except json.JSONDecodeError:
            with open(file_path, "r", encoding="utf-8") as f:
                lines = f.readlines()

            fixed_content = "[\n" + ",\n".join(line.strip() for line in lines if line.strip()) + "\n]"
            file_fixed_path = file_path.replace(".json", "_fixed.json")

            with open(file_fixed_path, "w", encoding="utf-8") as f:
                f.write(fixed_content)

            with open(file_fixed_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            print(f"✅ Archivo corregido guardado en: {file_fixed_path}")

        df = pd.DataFrame(data)

        # 🍽️ Filtrar restaurantes
        df = df[df['category'].apply(lambda x: any('restaurant' in cat.lower() for cat in x) if isinstance(x, list) else False)]

        # 🧹 Eliminar columnas con muchos nulos
        umbral_nulos = 0.40
        nulos = df.isnull().mean()
        columnas_nulas = nulos[nulos > umbral_nulos].index.tolist()
        df = df.drop(columns=columnas_nulas)

        # 🔁 Eliminar duplicados
        df_temp = df.applymap(lambda x: str(x) if isinstance(x, (list, dict)) else x)
        df = df[~df_temp.duplicated()]

        # 🗺️ Extraer código de estado y ciudad
        df['state_code'] = df['address'].apply(extraer_estado)
        df['city_name'] = df['address'].apply(extraer_ciudad)

        # 🧪 Normalizar tipos de datos
        df['name'] = df['name'].astype('string')
        df['address'] = df['address'].astype('string')
        df['gmap_id'] = df['gmap_id'].astype('string')
        df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
        df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
        df['category'] = df['category'].astype('string')
        df['avg_rating'] = pd.to_numeric(df['avg_rating'], errors='coerce')
        df['num_of_reviews'] = pd.to_numeric(df['num_of_reviews'], errors='coerce').astype('Int64')
        df['state_code'] = df['state_code'].astype('string')
        df['city_name'] = df['city_name'].astype('string')

        # 🧹 Eliminar columnas innecesarias
        columnas_a_eliminar = ['MISC', 'url', 'state', 'relative_results','price']
        df = df.drop(columns=[col for col in columnas_a_eliminar if col in df.columns])
        print("✅ Columnas eliminadas:", columnas_a_eliminar)

        # 📥 Agregar al listado de DataFrames
        lista_df.append(df)

# ────────────────────────────────
# 🧩 Unir todos los DataFrames
# ────────────────────────────────
df_total = pd.concat(lista_df, ignore_index=True)

# 📍 Filtrar por estados: California y Wyoming
df_filtrado = df_total[df_total['state_code'].isin(['CA', 'WY'])]

# 📊 Resultado final
print("✅ Total de restaurantes en CA y WY:", df_filtrado.shape[0])
print(df_filtrado[['name', 'state_code', 'city_name']].head())

# 💾 Guardar como Parquet
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df_filtrado.to_parquet(output_path, index=False)
print(f"💾 Archivo guardado en: {output_path}")


📂 Procesando: 1.json
✅ Archivo corregido guardado en: C:\Users\yanin\OneDrive\Desktop\etl\data_lake\raw\google\metadata-sitios-20250406T191417Z-001\metadata-sitios\1_fixed.json


C:\Users\yanin\AppData\Local\Temp\ipykernel_5428\1539381547.py:74: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_temp = df.applymap(lambda x: str(x) if isinstance(x, (list, dict)) else x)


✅ Columnas eliminadas: ['MISC', 'url', 'state', 'relative_results', 'price']
📂 Procesando: 10.json
✅ Archivo corregido guardado en: C:\Users\yanin\OneDrive\Desktop\etl\data_lake\raw\google\metadata-sitios-20250406T191417Z-001\metadata-sitios\10_fixed.json


C:\Users\yanin\AppData\Local\Temp\ipykernel_5428\1539381547.py:74: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_temp = df.applymap(lambda x: str(x) if isinstance(x, (list, dict)) else x)


✅ Columnas eliminadas: ['MISC', 'url', 'state', 'relative_results', 'price']
📂 Procesando: 11.json
✅ Archivo corregido guardado en: C:\Users\yanin\OneDrive\Desktop\etl\data_lake\raw\google\metadata-sitios-20250406T191417Z-001\metadata-sitios\11_fixed.json


...

✅ Columnas eliminadas: ['MISC', 'url', 'state', 'relative_results', 'price']
📂 Procesando: 8.json
✅ Archivo corregido guardado en: C:\Users\yanin\OneDrive\Desktop\etl\data_lake\raw\google\metadata-sitios-20250406T191417Z-001\metadata-sitios\8_fixed.json


C:\Users\yanin\AppData\Local\Temp\ipykernel_5428\1539381547.py:74: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_temp = df.applymap(lambda x: str(x) if isinstance(x, (list, dict)) else x)


✅ Columnas eliminadas: ['MISC', 'url', 'state', 'relative_results', 'price']
📂 Procesando: 9.json
✅ Archivo corregido guardado en: C:\Users\yanin\OneDrive\Desktop\etl\data_lake\raw\google\metadata-sitios-20250406T191417Z-001\metadata-sitios\9_fixed.json


C:\Users\yanin\AppData\Local\Temp\ipykernel_5428\1539381547.py:74: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_temp = df.applymap(lambda x: str(x) if isinstance(x, (list, dict)) else x)


✅ Columnas eliminadas: ['MISC', 'url', 'state', 'relative_results', 'price']
✅ Total de restaurantes en CA y WY: 27074
                   name state_code    city_name
0          San Soo Dang         CA  Los Angeles
1          Vons Chicken         CA    La Mirada
12      TACOS LA CABANA         CA      Oakland
13  Mariscos el poblano         CA      Oakland
19         Off The Hoof         CA    Santa Ana
💾 Archivo guardado en: c:\Users\yanin\OneDrive\Desktop\etl\data_lake\trusted\google\restaurantes_CA_WY.parquet


In [2]:
import pandas as pd

# Ruta del archivo parquet
archivo_parquet = r"c:\Users\yanin\OneDrive\Desktop\etl\data_lake\trusted\google\restaurantes_CA_WY.parquet"

# Cargar el archivo
df = pd.read_parquet(archivo_parquet)

# Ver las primeras filas
print(df.head())


                  name                                            address  \
0         San Soo Dang  San Soo Dang, 761 S Vermont Ave, Los Angeles, ...   
1         Vons Chicken  Vons Chicken, 12740 La Mirada Blvd, La Mirada,...   
2      TACOS LA CABANA  TACOS LA CABANA, 2015 22nd Ave, Oakland, CA 94606   
3  Mariscos el poblano  Mariscos el poblano, 5401-5441 Coliseum Way, O...   
4         Off The Hoof    Off The Hoof, 201 E 4th St, Santa Ana, CA 92701   

                                 gmap_id   latitude   longitude  \
0  0x80c2c778e3b73d33:0xbdc58662a4a97d49  34.058092 -118.292130   
1  0x80dd2b4c8555edb7:0xfc33d65c4bdbef42  33.916402 -118.010855   
2  0x808f879f35b5088b:0xe3541cec7a95bd88  37.789076 -122.233884   
3  0x808f87f90c1f661f:0xf384e804a61e0c0b  37.764203 -122.214647   
4  0x80dcd95d192d988b:0x68795f58e35bf888  33.748329 -117.866045   

                category  avg_rating  num_of_reviews  \
0  ['Korean restaurant']         4.4              18   
1         ['Restaurant

In [3]:
# Mostrar cantidad y porcentaje de nulos por columna
nulos = df.isnull().sum().to_frame(name='nulos')
nulos['%'] = (df.isnull().mean() * 100).round(2)
nulos = nulos.sort_values(by='%', ascending=False)

print(nulos)


                nulos     %
hours            2416  8.92
address             0  0.00
name                0  0.00
gmap_id             0  0.00
latitude            0  0.00
category            0  0.00
longitude           0  0.00
avg_rating          0  0.00
num_of_reviews      0  0.00
state_code          0  0.00
city_name           0  0.00
